In [5]:
from sandp500.get_common_sp500_stock_tickers import get_common_sp500_stock_tickers
import pandas_datareader as pdr

ImportError: cannot import name 'get_common_sp500_stock_ticker' from 'sandp500.get_common_sp500_stock_tickers' (/Users/zhengmingsong/Documents/GitHub/arbitraryElasticNet/sandp500/get_common_sp500_stock_tickers.py)

In [2]:
from datetime import date
from dateutil.relativedelta import relativedelta
import pandas as pd

from time import sleep
import random


n_year=5
end_date = date.today()
start_date = end_date - relativedelta(years=n_year)
common_sp500_tickers = get_common_sp500_stock_tickers(start_date,end_date)
common_sp500_tickers = [x.replace(".","_") for x in common_sp500_tickers]

In [4]:
def download_stock(args):
    stock, start_date, end_date = args
    sleep(random.randint(0,3000)/1000)
    try:
#         print(stock)
        output_name = './individual_stock/'+stock + '_data.csv'
        rder = pdr.stooq.StooqDailyReader(stock, start=start_date, end=end_date)
        # rder = pdr.tiingo.TiingoDailyReader(stock, start=start_date, end=end_date, api_key='dc356db40532fd464d0ebaf06dd9d114fb49db1b')
        stock_df = rder.read()
        stock_df.to_csv(output_name)
        rder.close()
        return ''
    except Exception as e: 
        bad_names.append(stock)
        print('bad: {}, {}'.format(stock, e))   
        return stock

In [7]:
from datetime import datetime
from multiprocessing import Pool, Array
from itertools import repeat
import tqdm


now_time = datetime.now()
bad_names =[] #to keep track of failed queries

max_workers = 10

workers = min(max_workers, len(common_sp500_tickers)) #in case a smaller number of stocks than threads was passed in
p = Pool(max_workers)
bad_names = list(tqdm.tqdm(p.imap_unordered(download_stock, zip(common_sp500_tickers, repeat(start_date), repeat(end_date))), total=len(common_sp500_tickers)))


""" Save failed queries to a text file to retry """
if len(bad_names) > 0:
    with open('failed_queries.txt','w') as outfile:
        for name in bad_names:
            outfile.write(name+'\n')

#timing:
finish_time = datetime.now()
duration = finish_time - now_time
minutes, seconds = divmod(duration.seconds, 60)
print('getSandP_threaded.py')
print(f'The threaded script took {minutes} minutes and {seconds} seconds to run.')
#The threaded script took 0 minutes and 31 seconds to run.

 17%|█▋        | 67/393 [00:25<01:28,  3.66it/s]

bad: CPRI, (Status 404) (Quandl Error QECx02) You have submitted an incorrect Quandl code. Please check your Quandl codes and try again.


 50%|████▉     | 196/393 [01:05<00:49,  4.02it/s]

bad: JEF, (Status 404) (Quandl Error QECx02) You have submitted an incorrect Quandl code. Please check your Quandl codes and try again.


 55%|█████▍    | 215/393 [01:12<00:55,  3.20it/s]

bad: LIN, (Status 404) (Quandl Error QECx02) You have submitted an incorrect Quandl code. Please check your Quandl codes and try again.


100%|██████████| 393/393 [02:15<00:00,  1.83it/s]

getSandP_threaded.py
The threaded script took 2 minutes and 15 seconds to run.


In [9]:
# pd.core.common.is_list_like = pd.api.types.is_list_like
# import pandas_datareader.data as web
# sp500inx = web.DataReader('^GSPC', 'yahoo', start_date, end_date)

# sp500inx['Date'] = sp500inx.index
# sp500inx = pd.read_csv('sp500_index.csv')
download_stock(['d:^spx', start_date, end_date])
sp500inx = pd.read_csv('./individual_stock/^SPX_data.csv')
sp500inx.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,4/14/14,1818.180054,1834.189941,1815.800049,1830.609985,1830.609985,3111540000
1,4/15/14,1831.449951,1844.020020,1816.290039,1842.979980,1842.979980,3736440000
2,4/16/14,1846.010010,1862.310059,1846.010010,1862.310059,1862.310059,3155080000
3,4/17/14,1861.729980,1869.630005,1856.719971,1864.849976,1864.849976,3341430000
4,4/21/14,1865.790039,1871.890015,1863.180054,1871.890015,1871.890015,2642500000


In [12]:
sp500inx['Date'] = pd.to_datetime(sp500inx['Date'], format='%Y-%m-%d')

In [13]:
sp500inx['Date'] = sp500inx['Date'].dt.date
sp500inx = sp500inx[['Date', 'Close']]
sp500inx.columns=['Date', 'SP500']

In [14]:
sp500inx = sp500inx.reset_index(drop=True)

,Date,SP500
0,2014-04-14,1830.609985
1,2014-04-15,1842.979980
2,2014-04-16,1862.310059
3,2014-04-17,1864.849976
4,2014-04-21,1871.890015


In [15]:

import glob
csv_files = glob.glob('raw_eod_data/*_data.csv')
file = csv_files[0]
stock_name = file.split('/')[1].split('_')[:-1]
stock_name = '_'.join(stock_name)
df = pd.read_csv(csv_files[0])[['date', 'adjClose']]
df.columns = ['Date', stock_name]
for f in csv_files[1:]:
#     df = pd.merge(df, pd.read_csv(f)[['Date', 'Adj. Close']], how='inner', on='Date')
#     print(df.shape)
    df_ = pd.read_csv(f)[['date', 'adjClose']]
    stock_name = f.split('/')[1].split('_')[:-1]
    stock_name = '_'.join(stock_name)
    df_.columns = ['Date', stock_name]
    if df_.shape[0]>=990:
        df = pd.merge(df, df_, how='left', on='Date')
    else:
        print('skip {} as the shape of the data is ({}, {}), which is less that 995 data point'.format(f.split('_')[0], df_.shape[0], df_.shape[1]))

In [35]:
# df.Date[0]
df['Date'] = pd.to_datetime(df['Date'])
df['Date'] = df['Date'].dt.date

In [37]:
df = pd.merge(df, sp500inx, how='inner', on='Date')
df = df.set_index('Date', drop=True)

In [40]:
pct_df = df.pct_change(fill_method='ffill')
pct_df = pct_df.iloc[1:]
pct_df.to_csv('sp500_pct.csv')

0


,XRX,GS,MTB,V,CTAS,APH,BBT,RHI,HD,PEP,...,HAL,CERN,SJM,WY,WFC,QCOM,COF,COG,F,SP500
Date,,,,,,,,,,,,,,,,,,,,,
2014-04-15,-0.003524,0.001163,-0.003806,0.015124,0.000533,0.001535,0.007981,0.002993,-0.001053,0.004907,...,0.015064,0.020806,-0.003250,-0.004661,0.013926,0.004423,0.006211,0.006410,0.008917,0.006757
2014-04-16,0.016799,0.014846,0.010383,0.026023,0.011005,0.022110,0.004853,0.032082,0.009092,0.009647,...,0.008838,0.000185,0.018406,0.005043,0.006355,0.008680,0.008588,0.032757,0.014520,0.010488
2014-04-17,-0.006087,0.001399,-0.000822,-0.006735,0.011763,0.007925,-0.035841,0.016386,0.006660,0.009201,...,0.006612,0.000371,-0.001343,-0.001075,-0.003259,0.014218,-0.003592,0.025551,-0.004356,0.001364
2014-04-21,0.003500,0.002287,-0.003044,0.005674,-0.004164,0.002550,-0.003427,0.000948,0.011286,0.004208,...,0.033169,0.002037,0.000310,0.011841,0.003883,-0.004796,0.002136,0.005727,-0.001250,0.003775
2014-04-22,0.010462,0.016857,0.006437,0.004017,0.007144,0.006889,0.008201,0.018238,0.021934,-0.008963,...,0.007311,-0.007392,0.002068,0.008865,0.002239,-0.003954,0.000799,0.015091,0.007509,0.004092


In [41]:
# import seaborn as sns
# pct_df['Date'] = pct_df.index
# sns.lineplot(data=pct_df.melt('Date', var_name='tickers', value_name='vals'), x='Date', y='vals', hue='tickers')


